In [ ]:
%load_ext autoreload
%autoreload

import os
import sys
from pathlib import Path
module_path = os.path.join(os.getcwd(),"Dropbox\Sch\SC4010 CRYPTOGRAPHY\Proj\kyber\src") #TODO: CHANGE PATH TO SOURCE CODE
sys.path.insert(0, module_path)

In [2]:
from kyber_kem import kyber_kem
from kyber_kem import *
import itertools

## KYBER-512 demo ##

Kyber 512 is a key encapsulation mechanism that shares a secret 256-bit key. Its offers 128 bits of random bit generator strength.

Key generation: Alice generates her secret key, and sends over the public parameters rho and t.

In [3]:
KYBER_512 = kyber_kem(3329, 256, 2, 2, 2, 12)
ek, dk = KYBER_512.kem_keygen()
rho, t = ek
print("PUBLIC KEY: Alice sends this to Bob")
print_bytes(rho + t)



PUBLIC KEY: Alice sends this to Bob
b'68142bff435c218002bc30f97746e1402f38b49095cc051d29ee6ad09d0cea2f94310cc1530cdd1c6dd46419a48927ef022eded57814a3c4f177927c595f4a2a30e91188e4b8351f2c652b4a6acb347ce6781027324a3fa5a25cc225347ac2ac63a87111c5c725b42481cae6e8af111184f4509b6a504e0ad4be89c47001438addd15479b08acca4a02a2c8cb955a5a58567a176b4b7ecbbe0477e25e73cfa215c41923d1ada301ca04ee8d1849708449d479ea8184f1eb073f50b1a30b11f13195f8d5485f79bc5c4b0392c33046c665b3eeb7b48d87e06623f1ed64ee7e08846e21959460cfad6a30e5a7a26d8a8bd1971e480bcf1702c83b29cfa2990f345b10cd3358f6655201187f2bba13b780e27fa258cf30b87e34f01f41d02e507a9a89082a1a4b545196c3cb25ec31645fb4e9060cae914b11ae58d4f42b824c0556564b79510248b8141144370c2a0b94d7bc5ddf86f3c679ce58622b6174eb8370371d00373778e2cf267cd21a7c3627619c0b856b692062c1461514284ac90c5a3512c2b4b6a27906f599c78b974f58bc6dde61546f87f078cabd8682251a9032b631be361637637a5a14ca1506828e69184a5610e3d79cdb81335b210831674ab221226c2065bda08bfc8c99da45278cc6417bd1035e41b738c3c3e6f8c67bff6

Encapsulation: Bob generates a secret message m and derives the shared key K from it. From Alice's public parameters he derives the ciphertext, consisting of u and v. 

In [4]:

u_bytes, v_bytes, K = KYBER_512.kem_encapsulate(ek)
print("CIPHERTEXT: Bob sends this to Alice")
print_bytes(u_bytes + v_bytes)

CIPHERTEXT: Bob sends this to Alice
b'd5b6cdb77c6b9ff276add7036147718245c932d3a7b9c4b6d1d59d23f3acdccc7a92f9bc2fb325d6ec8452932c4e118987c1bfaf404704eb68de291e77578cea1a8d0f8a196ea891c3ab1b7ee6cc7fba04dd912fb51179e68b226452b8f4049854654ce2795808e75f9b496e277699887b458a16a57b964b856a5c29a354fc0258dda603e3c97aaeb82f89769b2ce52cfb8ab87db48a9271270aa786b9719849938341573d77918a6c88a843016cd8ecb1c78bab819ac51e25cbbd385f4ea7b3f9cc114c66607545149f439c0afba17da00948c56992f2b01a8c8c7587844bca15c258abb776696fb95184bba2cee60ace138e247523f6793e9f403950f08c4f30346c90ac30e6573791c77a2850c720b049900e1ccb7711250e6eb4a5c2d80a7ec73f134bce76262707ea982c55c1d5fb1a3a7b41c02003ae447b04b37fd1ea2072ab535c79290b735b8fb8b3f57a2c36398ec2c10caa3a138e12abb694714f408465f5bae8014ec33851554382393480e2a253276b3aba7815d0f54e50db266131ca5f851a7dd728c2d78ec5d3ceafa558d96a8e40e50da1497011a8589785aeca462b633175177c393a774d605b30ffc02b2703715454b7a64b0ffd4a30d2545a8c19ce528366429b528036768cc8b35f98c7293823dc003da90368b2563284

In [5]:
print("SECRET SHARED KEY GENERATED BY BOB: ")
print_bytes(K)

SECRET SHARED KEY GENERATED BY BOB: 
b'9bf74a476277f6175ea88dbaa898ceedf7026796511923b7db484f7c2fb58440'


Decapsulation: Alice decrypts the ciphertext and computes K_prime. She then checks that K_prime will give the correct ciphertext, meaning there has been no decapsulation failure. (The chance of decapsulation failure is very slim). If there has been decapsulation failure, a pre-generated garbage key is returned.

In [6]:

K_prime = KYBER_512.kem_decapsulate(u_bytes, v_bytes, dk)
print("SECRET SHARED KEY DECAPSULATED BY ALICE: ")
print_bytes(K_prime)

SECRET SHARED KEY DECAPSULATED BY ALICE: 
b'9bf74a476277f6175ea88dbaa898ceedf7026796511923b7db484f7c2fb58440'


## ATTACKS ##

### Bad RNG ###

Bob is too lazy to come up with a good 256-bit message to encrypt, and decides to fix the first 31 bytes of it. What's the worst that could happen? Nobody knows what he has done. And after all, we still have all that fancy schmancy quantum protection, right?

In [7]:
KYBER_512_BadRNG = kyber_kem(3329, 256, 2, 2, 2, 12)
ek1, dk1 = KYBER_512_BadRNG.kem_keygen()
rho1, t1 = ek1
print("PUBLIC KEY: Alice sends this to Bob")
print_bytes(rho + t)

PUBLIC KEY: Alice sends this to Bob
b'68142bff435c218002bc30f97746e1402f38b49095cc051d29ee6ad09d0cea2f94310cc1530cdd1c6dd46419a48927ef022eded57814a3c4f177927c595f4a2a30e91188e4b8351f2c652b4a6acb347ce6781027324a3fa5a25cc225347ac2ac63a87111c5c725b42481cae6e8af111184f4509b6a504e0ad4be89c47001438addd15479b08acca4a02a2c8cb955a5a58567a176b4b7ecbbe0477e25e73cfa215c41923d1ada301ca04ee8d1849708449d479ea8184f1eb073f50b1a30b11f13195f8d5485f79bc5c4b0392c33046c665b3eeb7b48d87e06623f1ed64ee7e08846e21959460cfad6a30e5a7a26d8a8bd1971e480bcf1702c83b29cfa2990f345b10cd3358f6655201187f2bba13b780e27fa258cf30b87e34f01f41d02e507a9a89082a1a4b545196c3cb25ec31645fb4e9060cae914b11ae58d4f42b824c0556564b79510248b8141144370c2a0b94d7bc5ddf86f3c679ce58622b6174eb8370371d00373778e2cf267cd21a7c3627619c0b856b692062c1461514284ac90c5a3512c2b4b6a27906f599c78b974f58bc6dde61546f87f078cabd8682251a9032b631be361637637a5a14ca1506828e69184a5610e3d79cdb81335b210831674ab221226c2065bda08bfc8c99da45278cc6417bd1035e41b738c3c3e6f8c67bff6

In [8]:
FIXED_BYTES = b'\x91\x98\xfc\x1e(\x0c\x84}V\x02jJ\x8a\x1a\xe7\x87\x1a\x8fI\\\xe7s\xd3u"[es\xbb\xd2\xac'

u_bytes1, v_bytes1, K1 = KYBER_512_BadRNG.kem_encapsulate(ek1, bad_rng=True, fixed_bytes = FIXED_BYTES)
print("CIPHERTEXT: Bob sends this to Alice")
print_bytes(u_bytes1 + v_bytes1)
print(K1)

CIPHERTEXT: Bob sends this to Alice
b'2dc3679d4b8c01625a01d1979bdba009168017eb394bb573f795827f7b365184a4300b79b8bb5be87b8ab2bb3c0bd223f1831ebe73c5e473217758ae6a0c3e214b1b488a908858846400a6b9aa6ade36c3ae205a43740d824b6110b6256767336adc3a67039a13e5cdf459ac93918cc9239e5ba575bda1c5f880301d4ca93bca8ecfc3ae5ce662499b3a80b8154344a00f264518162c1e833ca7925cec21c84e681b49362ed1b5c13c7bb01941746e6b73aab99932a813d582cd13865702f958f7971176d131de04a029350f9b1112a1d7c0e563428caa12b9ca454b43c6cd87a49bb9245a8a58112814f4f066c0d9326a671823f4955ef13bea8cb774830344643ab9b75f95d512301aa9f6b9a25928a623295e07c3b262b4226376358180655440758787908261bf36a173106415e4a4c0efc164a0292d00ab7dd4731270455195e73707384b349521076771a0c27ba3db1c8134009d43bf70915d72591150ca690c57676b46cf215921d8a9aef9d66da8f55b9c06a1c461b92770598c303537ec77e3899834aac349d5ca9af78ed7fc49fab7777914bd86eaaceb517b0d7a211fc9270ce1a3a6e562f98971178a6e4293ca34fb875480bd431121a1ab9c62244d01e1af992196eb3968dabc3d2806b564595741888acf4134eb1871c8527096

Unfortunately for Bob, Eve has managed to steal his fixed_bytes while being invisible. With 2 bytes of entrophy, she can easily brute force his shared secret key from his ciphertext!

In [13]:
def brute_force(ek1, u_bytes, v_bytes, fixed_bytes, possible_bytes):
    rho1, t1 = ek1
    for bytes_ in possible_bytes:
        possible_m = fixed_bytes+bytes_

        h = KYBER_512_BadRNG._H(rho1 + t1)
        K_prime, R_prime = KYBER_512_BadRNG._G(possible_m + h)
        u_prime, v_prime = KYBER_512_BadRNG.kem_encrypt(possible_m, ek1, R_prime)

        if(u_bytes == u_prime and v_bytes == v_prime):
            return K_prime
        # if(K_prime == K1):
        #     return K_prime


In [ ]:
%%time

byte_strings = itertools.product(range(256))
possible_bytes =  [bytes(b1) for b1 in byte_strings]
K_prime = brute_force(ek1, u_bytes1, v_bytes1, FIXED_BYTES, possible_bytes)
if(K_prime):
    print("SECRET SHARED KEY BRUTE FORCED BY EVE: ")
    print_bytes(K_prime)

SECRET SHARED KEY BRUTE FORCED BY EVE: 
b'31f7c2ee1ef3a7b4e44986f55d8f802e4d215bcb760e0364412d9176df98cd74'
CPU times: total: 5 s
Wall time: 5 s


In [11]:
print("SECRET SHARED KEY GENERATED BY BOB: ")
print_bytes(K1)

SECRET SHARED KEY GENERATED BY BOB: 
b'31f7c2ee1ef3a7b4e44986f55d8f802e4d215bcb760e0364412d9176df98cd74'


In [12]:
K_prime1 = KYBER_512.kem_decapsulate(u_bytes1, v_bytes1, dk1)
print("SECRET SHARED KEY DECAPSULATED BY ALICE: ")
print_bytes(K_prime1)

SECRET SHARED KEY DECAPSULATED BY ALICE: 
b'31f7c2ee1ef3a7b4e44986f55d8f802e4d215bcb760e0364412d9176df98cd74'
